In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# Do new simulations 

In [ ]:
from simulation import run_simulation

### Option 1: Reproduce results

#key = 'fixed_complexity'
#paramfolder = 'results/{}/'.format(key)
#results = run_simulation(paramfolder, outfolder=None)

### Option 2: Add new results to existing folder 

#run_simulation(paramfolder, outfolder=paramfolder)

### Option 3: Created completely new results or add new to existing folder.

parameters = {
    'key': 'fixed_complexity_right_inverse', 
    'n_its': 10, 
    'time': 'undefined', 
    'positions': [2,4,6,8,10,12,14],
    'complexities': [3,4,5,6], 
    'anchors': [8,9,10], 
}
outfolder = 'results/{}/'.format(parameters['key'])
run_simulation(parameters, outfolder, solver="rightInverseOfConstraints")

# Read and plot results

In [ ]:
from simulation import read_results, read_params
from global_variables import DIM
from plotting_tools import add_plot_decoration, generate_labels

key =  'fixed_complexity_right_inverse'
resultfolder = 'results/{}/'.format(key)
results = read_results(resultfolder + 'result_')
parameters = read_params(resultfolder + 'parameters.json')
complexities = np.array(parameters['complexities'])
positions = np.array(parameters['positions'])
anchors = np.array(parameters['anchors'])

    
dims = {
    'complexities' : 0,
    'anchors' : 1,
    'positions' : 2,
    'measurements' : 3,
}    

# Choose which parameters to plot - the values are indexes to the previously given parameters
indices = [[]]*len(dims)
indices[dims['complexities']] = range(len(complexities))
indices[dims['anchors']] = [0]
indices[dims['positions']] = [3]
indices[dims['measurements']] = range(np.max(positions[indices[dims['positions']]])*np.max(anchors[indices[dims['anchors']] ]) ) 

grid = np.meshgrid(indices[0], indices[1], indices[2], indices[3])
title, label = generate_labels(dims, parameters, indices)

print("NOTE: values are nan if never visited (white)")

plt.matshow(np.squeeze(results['successes'][grid]))
plt.title('Number of successful solutions'+title)
add_plot_decoration(label, parameters)
plt.show()

result_dimentions = np.shape(results['successes'])
bound_theory = np.zeros(result_dimentions)

for c_idx in indices[dims['complexities']]:
    for a_idx in indices[dims['anchors']]:
        for p_idx in indices[dims['positions']]:
            for m_idx in indices[dims['measurements']]:
                n_min = complexities[c_idx] * 2 + DIM * complexities[c_idx]
                max_measurements = positions[p_idx]*anchors[a_idx]
                if (m_idx <= max_measurements - n_min) and (positions[p_idx]>= complexities[c_idx]):
                    bound_theory[c_idx, a_idx, p_idx, m_idx] = 1.0
                if m_idx > max_measurements:
                    bound_theory[c_idx, a_idx, p_idx, m_idx] = np.nan

print("NOTE: shows theoretical bound () and value from which zero measurements are left (white)")
plt.matshow(np.squeeze(bound_theory[grid]))
plt.title('theoretical bound'+title)
add_plot_decoration(label, parameters)
plt.show()

plt.matshow(np.squeeze(results['num-not-solved'][grid]))
plt.title('Number of solver failures'+title)
add_plot_decoration(label, parameters)
plt.show()

plt.matshow(np.squeeze(results['num-not-accurate'][grid]))
plt.title('Number of inaccurate solutions'+title)
add_plot_decoration(label, parameters)
plt.show()